In [ ]:
import liesel.goose as gs
import numpy as np
import pymc as pm

from liesel.experimental.pymc import PyMCInterface

In [ ]:
RANDOM_SEED = 123
rng = np.random.RandomState(RANDOM_SEED)

In [ ]:
n = 100
p = 4

sigma_scalar = 1.0
beta_0_scalar = 1.0
beta_vec = [0.0, 0.0, 1.0, 2.0]

In [ ]:
X = rng.uniform(size=(n, p))

errors = rng.normal(size=n)

y = X @ beta_vec + sigma_scalar * errors

In [ ]:
spike_and_slabb_model = pm.Model()

mu = 0.

alpha_tau = 0.01
beta_tau = 0.01

alpha_sigma = 0.01
beta_sigma = 0.01

alpha_theta = 1.0
beta_theta = 1.0

p_theta = 0.5

nu = 0.0001

with spike_and_slabb_model:
    # priors    
    sigma = pm.InverseGamma(
        "sigma", alpha=alpha_sigma, beta=beta_sigma
    )

    theta = pm.Beta("theta", alpha=alpha_theta, beta=beta_theta)
    delta = pm.Bernoulli("delta", p=theta)
    spike = pm.InverseGamma("spike", alpha=alpha_tau, beta=nu * beta_tau)
    slab = pm.InverseGamma("slab", alpha=alpha_tau, beta=beta_tau)
    tau = (1 - delta) * spike + delta * slab

    beta = pm.Normal("beta", mu=0.0, sigma=tau, shape=p)

    # likelihood
    pm.Normal("y", mu=X @ beta, sigma=sigma, observed=y)

spike_and_slabb_model

In [ ]:
spike_and_slabb_model.unobserved_value_vars

In [ ]:
interface = PyMCInterface(spike_and_slabb_model, additional_vars=["sigma", "theta"])
state = interface.get_initial_state()

In [ ]:
builder = gs.EngineBuilder(seed=1, num_chains=4)
builder.set_model(interface)
builder.set_initial_values(state)
builder.set_duration(warmup_duration=1000, posterior_duration=2000)

builder.add_kernel(gs.NUTSKernel(["beta", "sigma_log__"]))

builder.positions_included = ["sigma"]

engine = builder.build()

engine.sample_all_epochs()

In [ ]:
results = engine.get_results()
print(gs.Summary(results))